In [1]:
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
import json
load_dotenv(override=True)

True

In [3]:
client=OpenAI()
chit=0
bhit=0


In [37]:
def Chhannel1(question):
    global bhit
    print("Bollywood Crazy")
    bhit +=1
    resp=client.chat.completions.create(model="gpt-4o-mini",messages=[{"role":"system","content":"You are a bollywood expert. You have extreme knowledge of bollywood,celebrities,boxoffice,movies,latest releast etc etc.Please Answer question factualy correct and concise"}
    ,{"role":"user","content":question}])

    return resp.choices[0].message.content



In [38]:
def Chhannel2(question):
    global chit
    print("Cricket Mania")
    chit +=1
    resp=client.chat.completions.create(model="gpt-4o-mini",messages=[{"role":"system","content":"You are a Cricket Fanatic.You know about the sports like anything. You have extreme knowledge of cricket matches, players statistics, records,schedule and is not confined to only test but onday and T20 as well including regional tournament. You represnt THE International cricket council voice.Please Answer question factualy correct and concise"}
    ,{"role":"user","content":question}])
    
    return resp.choices[0].message.content



In [39]:
def UnknownQuestion(question):
    msg="this topic is unknown to me, Please ask question related to bollywood and cricket only  "
    return msg

In [51]:
Chhannel1_json={"name":"Chhannel1"
                ,"description":"Bollywood chhanel covering all news related to bollywood "
                ,"parameters":{
                    "type":"object",
                    "properties" :{
                        "question":
                                {"type":"string",
                                "description":"User question"
                                }
                    }
                    ,"required":["question"]
                }
                }

In [52]:
Chhannel2_json={"name":"Chhannel2"
                ,"description":"Cricket chhanel covering all news related to Cricket sport "
                ,"parameters":{
                    "type":"object",
                    "properties" :{
                        "question":
                                {"type":"string",
                                "description":"User question"
                                }
                    }
                    ,"required":["question"]
                }
                }

In [53]:
UnknownQuestion_json={"name":"UnknownQuestion"
                ,"description":"Fallback for bollywood/cricket queries "
                ,"parameters":{
                    "type":"object",
                    "properties" :{
                        "question":
                                {"type":"string",
                                "description":"User question"
                                }
                    }
                    ,"required":["question"]
                }
                }

In [54]:
tools=[{"type":"function","function":Chhannel1_json},
    {"type":"function","function":Chhannel2_json},
    {"type":"function","function":UnknownQuestion_json}]

In [44]:
tools

[{'type': 'function',
  'function': {'name': 'Chhannel2',
   'description': 'Bollywood chhanel covering all news related to bollywood ',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'User question'}},
    'required': ['question']}}},
 {'type': 'function',
  'function': {'name': 'Chhannel2',
   'description': 'Cricket chhanel covering all news related to Cricket sport ',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'User question'}},
    'required': ['question']}}},
 {'type': 'function',
  'function': {'name': 'UnknownQuestion',
   'description': 'Fallback for bollywood/cricket queries ',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'User question'}},
    'required': ['question']}}}]

In [55]:
def handle_tool_calls(tool_calls):
    results=[]
    for tool_call in tool_calls:
        toolfunc=globals().get(tool_call.function.name)
        #print("hi")
        #print(toolfunc)
        args=json.loads(tool_call.function.arguments or "{}")
        print(f"tool called : {toolfunc}")
        out=toolfunc(**args) if toolfunc else ""
        results.append({"role":"tool","tool_call_id":tool_call.id,"content":json.dumps({"result": out})})
    return results

In [56]:
# 4) Nudge routing with a clearer system instruction
SYSTEM = (
    "Choose exactly one tool based on the user question: "
    "Channel1 for Bollywood topics (movies, actors ,BOX OFFICE,latest release, actor profile) "
    "Channel2 for Cricket topics (players, matches, IPL/ODI/T20/Tests), "
    "otherwise UnknownQuestion. Pass the user's question string as 'question'. "
    "After tools, provide the final concise answer."
)

In [57]:



def ask(user_question: str, tool_choice="auto"):
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": user_question}
    ]
    choice_arg = tool_choice  # "auto" lets the model pick the correct tool

    while True:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice=choice_arg
        )
        ch = resp.choices[0]
        if ch.finish_reason == "tool_calls":
            messages.append(ch.message)
            messages.extend(handle_tool_calls(ch.message.tool_calls))
            choice_arg = "auto"
            continue
        return ch.message.content

# ------- Minimal Gradio UI -------


In [58]:
# Demo
print(ask("What is akshay kumar latest release "))
print(ask("Virat kohlis debut"))

tool called : <function Chhannel1 at 0x125032980>
Bollywood Crazy
As of October 2023, Akshay Kumar's latest release is the film "Bachchhan Paandey," which premiered on March 18, 2022. Please verify for any new releases or announcements that may have occurred after that date.
tool called : <function Chhannel2 at 0x125033ce0>
Cricket Mania
Virat Kohli made his international debut for India on August 18, 2008, in a One Day International (ODI) match against Sri Lanka in Dambulla. He made his Test debut on July 21, 2011, against the West Indies in Kingston. Kohli has since become one of the most successful and celebrated cricketers in history.


In [59]:
demo = gr.Interface(
    fn=lambda q: ask(q, tool_choice="auto"),
    inputs=gr.Textbox(label="Your question", lines=2, placeholder="Ask about Bollywood or Cricket"),
    outputs=gr.Textbox(label="Answer"),
    title="Bollywood/Cricket Router (Tools)",
    description="Type a question; the model picks the right tool and answers."
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


tool called : <function Chhannel1 at 0x125032980>
Bollywood Crazy
tool called : <function UnknownQuestion at 0x125033ba0>
tool called : <function Chhannel2 at 0x125033ce0>
Cricket Mania


In [ ]:
# Minimal Gradio UI that also shows channel hit counts as Markdown
import gradio as gr

def ask_with_hits(q: str):
    ans = ask(q, tool_choice="auto")
    try:
        b = bhit
    except NameError:
        b = 0
    try:
        c = chit
    except NameError:
        c = 0
    hits_md = f"""
### Channel hits
- Bollywood (Chhannel1): {b}
- Cricket (Chhannel2): {c}
"""
    return ans, hits_md

hits_demo = gr.Interface(
    fn=ask_with_hits,
    inputs=gr.Textbox(label="Your question", lines=2, placeholder="Ask about Bollywood or Cricket"),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Markdown(label="Channel Hits")
    ],
    title="Bollywood/Cricket Router (Tools) + Hits",
    description="Model routes to the correct tool; Markdown shows channel hit counts."
)

hits_demo.launch()


* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


tool called : <function Chhannel1 at 0x125032980>
Bollywood Crazy
tool called : <function UnknownQuestion at 0x125033ba0>
